
# Projet: Facebook BaBi tasks
Notebook de la semaine du 22/03
Par Thierry Loesch et Bryce Tichit

Comme demandé par notre tuteur, nous allons désormais essayer d'adapter les travaux fait jusqu'ici sur un autre projet sensiblement identique.

-----------------------------------

5.1  Notation automatique de réponses courtes à des questions

Les données sont ici pour le téléchargement et leur description est dans cet article

Lorsque l'on connait:

    une question,
    une réponse d'un étudiant,
    la bonne réponse, 

comment prédire la note à donné à la réponse de l'étudiant. Il peut être aussi intéressant d'essayer d'apprendre à générer la réponse. 

--------------------------

On cherchera en premier à prédire la note de l'étudiant, le réseau devra rendre un réel. Normaliser les notes (les mettre entre 0 et 1) et activation par Sigmoid (output entre 0 et 1) afin de prédire la note.


TODO:

- Parser données
- Vectorisation
- Modèle

In [48]:
import re
from keras.utils.data_utils import get_file
import zipfile
import string

def tokenize(sent):
    
    def remPunctuation(sent):
        return "".join(l for l in sent if l not in string.punctuation)
        
    return [x.strip() for x in re.split('(\W+)?', remPunctuation(sent.lower())) if x.strip()]

def parseData(questions,answers,student_answers=''):
    
    #questions_data = dict() #ce dictionnaire contiendra les tuples (question,answer) indexés par la clé "numéro de question"
    
    questions_data = dict()
    
    for line in questions.split('\n'): #une ligne = une question
        line = line.decode('utf-8').strip()
        try:
            index,question = line.split(' ',1)
        except:
            continue
        
        questions_data[index] = [question.replace('<STOP>',''),None]
    
    for line in answers.split('\n'):
        line = line.decode('utf-8').strip()
        
        try:
            index,answer = line.split(' ',1)
        except:
            continue
            
        questions_data[index][1] = answer.replace('<STOP>','')
        
    questions_data = {k: tuple(map(tokenize,v)) for k, v in questions_data.items()}
    
    stud_ans = dict()
    
    for line in student_answers.split('\n'):
        line = line.decode('utf-8')
        try:
            index,ans = line.split(' ',1)
        except:
            continue
            
        answers=ans.split('<STOP>')
        answers=[a for a in answers if a]
            
        if index in stud_ans:
            stud_ans[index] += map(tokenize,answers)
        else:
            stud_ans[index] = map(tokenize,answers)
    
    return questions_data,stud_ans
        
    
def parseIntoExamples(qd,sa):
    
    ret = list()
    
    for index,q_a in qd.items():
        scores = [k for k in archive.read('data/scores/'+str(index+'/ave')).split('\n') if k]
        for stud_ans,score in zip(sa[index],scores):
            try:
                ret.append((q_a[0],q_a[1],stud_ans,float(score)/5.))
            except:
                print "Erreur float.Score = {} ".format(score,stud_ans)
    return ret
                       


    

In [49]:
try:
    path = get_file('shortAnswerGrading-v-2-0.zip',origin='http://web.eecs.umich.edu/~mihalcea/downloads/ShortAnswerGrading_v2.0.zip')
except:
    print "error while downloading file"
    
    
archive = zipfile.ZipFile(path,'r')
    
questions = archive.read('data/sent/questions') 
answers = archive.read('data/sent/answers')
student_answers = archive.read('data/sent/all') 

dic1,dic2 = parseData(questions,answers,student_answers)


data = parseIntoExamples(dic1,dic2)

#print data[0]

vocab = set()
for (a,b,c,_) in data:
    for x in [a,b,c]:
        vocab.update(x)
vocab=sorted(list(vocab))

print len(vocab)

vocab_size = len(vocab)
word_idx = dict((c,i+1) for i,c in enumerate(vocab))
question_maxsize = max((len(x) for x,_,_,_ in data))
answer_stud_maxsize = max(len(x) for _,_,x,_ in data)
answer_maxsize = max((len(x) for _,x,_,_ in data))

vocab[0:300]


2223


[u'0',
 u'000000',
 u'012',
 u'0x',
 u'1',
 u'10',
 u'115',
 u'12345',
 u'123456789',
 u'124',
 u'154',
 u'18',
 u'1each',
 u'1long',
 u'1st',
 u'2',
 u'20',
 u'256',
 u'2nd',
 u'2the',
 u'3',
 u'35',
 u'3n',
 u'4',
 u'40',
 u'400',
 u'5',
 u'5555',
 u'5657',
 u'6',
 u'68',
 u'7',
 u'72',
 u'8',
 u'80',
 u'88123',
 u'8th',
 u'a',
 u'ability',
 u'able',
 u'about',
 u'above',
 u'abstract',
 u'abstraction',
 u'accepted',
 u'accepts',
 u'accesed',
 u'access',
 u'accessed',
 u'accessible',
 u'accessing',
 u'accessmodifier',
 u'accessspecifications',
 u'accessspecifiers',
 u'accomplish',
 u'according',
 u'accordingly',
 u'account',
 u'accurate',
 u'achieved',
 u'acknowledged',
 u'acordingly',
 u'across',
 u'act',
 u'acted',
 u'actions',
 u'activity',
 u'actual',
 u'actually',
 u'ad',
 u'add',
 u'added',
 u'adding',
 u'addition',
 u'additional',
 u'address',
 u'addressed',
 u'addresses',
 u'adds',
 u'adjacent',
 u'adjust',
 u'adjusted',
 u'adt',
 u'advance',
 u'advantage',
 u'advantages',
 u'

A faire:
- Nettoyer le vocabulaire pour diminuer sa taille, synonymes, lowercase


In [50]:
#Mettons tout ça dans des matrices
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def vectorize(data,wordidx,qmaxlen,amaxlen):
    X,Xs,Y = list(),list(),list()
    
    lookup = lambda m : wordidx[m]
    
    for question, _ , student_answer, score in data:
        X.append(map(lookup,question))
        Xs.append(map(lookup,student_answer))
        Y.append(score)
        
    return pad_sequences(X, maxlen=qmaxlen), pad_sequences(Xs, maxlen=amaxlen), np.array(Y)
        
    
X,Xq,Y = vectorize(data,word_idx,question_maxsize,answer_stud_maxsize)
#X_test,Xq_test,Y_test = vectorize(test,word_idx,story_max,question_max)
#X_val,Xq_val,Y_val = vectorize(validation,word_idx,story_max,question_max)

In [51]:
print X[0:10]
print Xq[0:10]
print Y[0:10]

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 2176 1040   38  832 1464]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 2176 1040   38  832 1464]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 2176 1040   38  832 1464]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 2176 1040   38  832 1464]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 2176 1040   38  832 1464]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  

In [61]:
#Model

from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge,RepeatVector,Activation
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.optimizers import RMSprop,Adam


embed_size = 50
batch_size=64
epochs=25

question_model = Sequential()
question_model.add(Embedding(vocab_size,embed_size,input_length=question_maxsize))

ans_model = Sequential()
ans_model.add(Embedding(vocab_size,embed_size,input_length=answer_stud_maxsize))
ans_model.add(LSTM(embed_size))
ans_model.add(RepeatVector(question_maxsize)) #permet d'ajuster la taille du modèle afin de préparer un merge


model = Sequential()
model.add(Merge([question_model, ans_model], mode='concat'))
model.add(LSTM(embed_size))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(optimizer=Adam(lr=0.01),loss='mse',metrics=['accuracy'])

model.summary()

history = model.fit([X, Xq], Y, batch_size=batch_size, nb_epoch=epochs)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_17 (Embedding)         (None, 35, 50)        111150                                       
____________________________________________________________________________________________________
embedding_18 (Embedding)         (None, 74, 50)        111150                                       
____________________________________________________________________________________________________
lstm_14 (LSTM)                   (None, 50)            20200                                        
____________________________________________________________________________________________________
repeatvector_8 (RepeatVector)    (None, 35, 50)        0                                            
___________________________________________________________________________________________

IndexError: index 2223 is out of bounds for size 2223
Apply node that caused the error: AdvancedSubtensor1(embedding_18_W, Reshape{1}.0)
Toposort index: 96
Inputs types: [TensorType(float32, matrix), TensorType(int32, vector)]
Inputs shapes: [(2223, 50), (4736,)]
Inputs strides: [(200, 4), (4,)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{2}(AdvancedSubtensor1.0, TensorConstant{[-1 50]})]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "<ipython-input-61-f1aaa27707d9>", line 18, in <module>
    ans_model.add(Embedding(vocab_size,embed_size,input_length=answer_stud_maxsize))
  File "/usr/lib/python2.7/site-packages/keras/models.py", line 114, in add
    layer.create_input_layer(batch_input_shape, input_dtype)
  File "/usr/lib/python2.7/site-packages/keras/engine/topology.py", line 341, in create_input_layer
    self(x)
  File "/usr/lib/python2.7/site-packages/keras/engine/topology.py", line 485, in __call__
    s